In [46]:
import pandas as pd
import matplotlib.pyplot as plt
import folium
import numpy as np
import seaborn as sns
import os
import fastparquet
import warnings
import geopy
from geopy.point import Point
import time
warnings.simplefilter(action='ignore', category=FutureWarning)

#df = pd.read_parquet('Downloads/airbnb_parquets')
#df.to_csv('csv_printout.csv')

In [57]:
def append_fig_to_html(list_of_figs):
    for fig in list_of_figs:
        with open("reports/report_draft.html",'a') as f:
            f.write(fig.to_html(full_html=False, include_plotlyjs='cdn'))

# write this later
"""
def overwrite_html(list_of_figs):
    
    with open("reports/report_draft.html",'w') as f:
    f.write(listing_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(price_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(price_v_numlist_fig.to_html(full_html=False, include_plotlyjs='cdn'))
"""

def calculate_mortgage(home_value, interest_rate, num_years, down_payment_pct=0):
    per_payment_interest = 0
    loan_value = 0

    if down_payment_pct >= 1:
        down_payment = down_payment_pct/100 * home_value
        loan_value = home_value - down_payment
    else:
        down_payment = down_payment_pct * home_value
        loan_value = home_value - down_payment

    if loan_value/home_value < 0.80: 
        # insert pmi calc here
        pmi = 0.01  # using near average value here
        pmi_cost = 0.0007*home_value    # shot in the dark after interpolating nerdwallet calculator
        if interest_rate >= 1:
            per_payment_interest = interest_rate/100/12
        else:
            per_payment_interest = interest_rate/12
        num_months = num_years*12
        mortgage = loan_value*(per_payment_interest*(1+per_payment_interest)**num_months)/((1+per_payment_interest)**num_months-1) + pmi_cost

        mortgage = np.round(mortgage, 2)
        return mortgage
    else:
        if interest_rate >= 1:
            per_payment_interest = interest_rate/100/12
        else:
            per_payment_interest = interest_rate/12
        num_months = num_years*12
        mortgage = loan_value*(per_payment_interest*(1+per_payment_interest)**num_months)/((1+per_payment_interest)**num_months-1)

        mortgage = np.round(mortgage, 2)
        return mortgage

def calculate_roi(airbnb_daily_price, occupancy_rate, monthly_mortgage, monthly_maintenence=0, monthly_taxes=0):
    gross_rev = airbnb_daily_price * occupancy_rate * 365/12
    net_rev = gross_rev - monthly_maintenence - monthly_taxes
    profit = net_rev - monthly_mortgage
    roi = profit/monthly_mortgage
    roi = roi
    return roi

def set_location_desc(df,lat,long,geolocator):
    start_time = time.time()
    lat = str(lat)
    print("---Lat casted at value at %s seconds ---" % (time.time() - start_time))

    long = str(long)
    print("---Long casted at %s seconds ---" % (time.time() - start_time))

    location = geolocator.reverse(lat+","+long)
    print("---Get location at %s seconds ---" % (time.time() - start_time))

    address = location.raw['address']
    print("---Get address at %s seconds ---" % (time.time() - start_time))

    city = address.get('city', '')
    print("---Get city value at %s seconds ---" % (time.time() - start_time))

    state = address.get('state', '')
    print("---Get state value at %s seconds ---" % (time.time() - start_time))

    country = address.get('country')
    print("---Get country value at %s seconds ---" % (time.time() - start_time))
    country_code = address.get('country_code')
    print("---Get country_code value at %s seconds ---" % (time.time() - start_time))
    return city

def set_city(lat, long, geolocator):
    start_time = time.time()
    lat = str(lat)
    print("---Latitude casted to string at %s seconds ---" % (time.time() - start_time))

    long = str(long)
    print("---Longitude casted at %s seconds ---" % (time.time() - start_time))

    location = geolocator.reverse(lat+","+long)
    print("---Get location json from geopy at %s seconds ---" % (time.time() - start_time))

    address = location.raw['address']
    print("---Get address json at %s seconds ---" % (time.time() - start_time))

    city = address.get('city', '')
    print("---Get city value at %s seconds ---" % (time.time() - start_time))
    return city

def set_state(lat, long):
    from geopy.geocoders import Nominatim
    geolocator = Nominatim(user_agent="geoapiExercises")
    lat = str(lat)
    long = str(long)
    location = geolocator.reverse(Point(lat,long))
    address = location.raw['address']
    state = address.get('state', '')
    return state

def set_country(lat, long):
    from geopy.geocoders import Nominatim
    geolocator = Nominatim(user_agent="geoapiExercises")
    lat = str(lat)
    long = str(long)
    location = geolocator.reverse(Point(lat,long))
    address = location.raw['address']
    country = address.get('country')
    return country

def set_country_code(lat, long):
    from geopy.geocoders import Nominatim
    geolocator = Nominatim(user_agent="geoapiExercises")
    lat = str(lat)
    long = str(long)
    location = geolocator.reverse(lat+","+long)
    address = location.raw['address']
    country_code = address.get('country_code')
    return country_code

def set_zipcode(lat, long):
    from geopy.geocoders import Nominatim
    geolocator = Nominatim(user_agent="geoapiExercises")
    lat = str(lat)
    long = str(long)
    location = geolocator.reverse(Point(lat,long))
    address = location.raw['address']
    zipcode = address.get('postcode', '')
    return zipcode


In [ ]:
"""
import os, json
import pandas as pd

# this finds our json files
path_to_json = '/Users/matthewgilgo/Downloads/airbnb_data/all_ids/all_ids.json/'
json_files = [pos_json for pos_json in os.listdir(path_to_json) if pos_json.endswith('.json')]

# here I define my pandas Dataframe with the columns I want to get from the json
jsons_data = pd.DataFrame(columns=['index', 'id', 'name', 'price', 'displayPrice','monthlyPriceFactor','weeklyPriceFactor','avgRating','reviewsCount','isNewListing','isSuperhost','lat','lng','personCapacity','size','beds','baths','title','property_type','town','top_lat','right_long','botton_lat','left_long','url'])

# we need both the json and an index number so use enumerate()
for ind, js in enumerate(json_files):
    with open(os.path.join(path_to_json, js)) as json_file:
        json_text = json.load(json_file)

        # here you need to know the layout of your json and each json has to have
        # the same structure (obviously not the structure I have here)
        index = json_text['features'][0]['index']
        id = json_text['features'][0]['id']
        name = json_text['features'][0]['name']
        price = json_text['features'][0]['price']
        displayPrice = json_text['features'][0]['displayPrice']
        monthlyPriceFactor = json_text['features'][0]['monthlyPriceFactor']
        weeklyPriceFactor = json_text['features'][0]['weeklyPriceFactor']
        avgRating = json_text['features'][0]['avgRating']
        reviewsCount = json_text['features'][0]['reviewsCount']
        isNewListing = json_text['features'][0]['isNewListing']
        isSuperhost = json_text['features'][0]['isSuperhost']
        lat = json_text['features'][0]['lat']
        lng = json_text['features'][0]['lng']
        personCapacity = json_text['features'][0]['personCapacity']
        size = json_text['features'][0]['size']
        beds = json_text['features'][0]['beds']
        baths = json_text['features'][0]['baths']
        title = json_text['features'][0]['title']
        property_type = json_text['features'][0]['property_type']
        town = json_text['features'][0]['town']
        top_lat = json_text['features'][0]['top_lat']
        right_long = json_text['features'][0]['right_long']
        botton_lat = json_text['features'][0]['botton_lat']
        left_long = json_text['features'][0]['left_long']
        url = json_text['features'][0]['url']          
        # here I push a list of data into a pandas DataFrame at row given by 'index'
        jsons_data.loc[ind] = [index,id,name,price,displayPrice,monthlyPriceFactor,weeklyPriceFactor,avgRating,reviewsCount,isNewListing,isSuperhost,lat,lng,personCapacity,size,beds,baths,title,property_type,town,top_lat,right_long,botton_lat,left_long,url]

# now that we have the pertinent json data in our DataFrame let's look at it
print(jsons_data)
"""

In [27]:
nc_dir = 'C:/Users/mattg/Desktop/Hobbies/airbnb_reports/bucket_data/listings/north_carolina'
first_file = ''
for file in os.listdir(nc_dir):
    first_file = os.path.join(nc_dir,file)
    break
listing_data = pd.read_parquet(first_file)
print(listing_data.shape)
for file in os.listdir(nc_dir):
    next_file = os.path.join(nc_dir,file)
    if next_file != first_file:
        next_listing = pd.read_parquet(next_file)
        listing_data = listing_data.append(next_listing)
print(listing_data.shape)

ne_dir = 'C:/Users/mattg/Desktop/Hobbies/airbnb_reports/bucket_data/listings/vt_nh'
for file in os.listdir(ne_dir):
    next_file = os.path.join(ne_dir,file)
    next_listing = pd.read_parquet(next_file)
    listing_data = listing_data.append(next_listing)
print(listing_data.shape)

#listing_data = pd.read_parquet('C:/Users/mattg/Desktop/Hobbies/airbnb_reports/bucket_data/listings/vt_nh')

(20, 25)
(19427, 25)
(35284, 25)


In [28]:
#listing_data = pd.read_parquet('C:/Users/mattg/Desktop/Hobbies/airbnb_reports/bucket_data/listings')
occ_data = pd.read_parquet('C:/Users/mattg/Desktop/Hobbies/airbnb_reports/bucket_data/occupancy')
price_data = pd.read_parquet('C:/Users/mattg/Desktop/Hobbies/airbnb_reports/bucket_data/prices')

In [60]:
# Clean up listing_data
conditions = [
    listing_data['baths'].str.contains('shared', na=False),
    listing_data['baths'].str.contains('Shared', na=False)
]

values = ['Shared', 'Shared']

listing_data['baths_type'] = np.select(conditions, values, default='Private')

conditions_loc = [
    listing_data['top_lat'] > 35.5
]

values_loc = ['New England']

listing_data['Region'] = np.select(conditions_loc, values_loc, default='Carolinas')


conditions_halfbath = [
    listing_data['baths'].str.contains('Half-bath', na=False),
    listing_data['baths'].str.contains('Shared half-bath', na=False),
    listing_data['baths'].str.contains('Private half-bath', na=False)
]

values_halfbath = [0.5,0.5,0.5]

listing_data['baths'] = np.select(conditions_halfbath, values_halfbath, default=listing_data['baths'])

listing_data['baths_no'] = listing_data['baths'].str.split(' ').str[0]
listing_data['beds_no'] = listing_data['beds'].str.split(' ').str[0]
listing_data['guest_no'] = listing_data['title'].str.split(' ').str[0]
listing_data['guest_no'] = listing_data['guest_no'].astype('float')
listing_data['id'] = listing_data['id'].astype('float64')
listing_data['id'] = listing_data['id'].astype('str')
listing_data['town'] = listing_data['town'].str.lower()
listing_data = listing_data[listing_data.lat.isna() == False]
#listing_data['city_geopy'] = listing_data.apply(lambda row: set_city(row['lat'], row['lng']), axis=1)
#listing_data['state_geopy'] = listing_data.apply(lambda row: set_state(row['lat'], row['lng']), axis=1)
#listing_data['country_geopy'] = listing_data.apply(lambda row: set_country(row['lat'], row['lng']), axis=1)
#listing_data['country_code_geopy'] = listing_data.apply(lambda row: set_country_code(row['lat'], row['lng']), axis=1)
#listing_data['zipcode_geopy'] = listing_data.apply(lambda row: set_zipcode(row['lat'], row['lng']), axis=1)

# Clean up occ_data
#occ_data['id'] = occ_data['id'].astype('int')
occ_data['id'] = occ_data['id'].astype('str')

# Clean up price_data
#price_data['id'] = price_data['id'].astype('int')
price_data['id'] = price_data['id'].astype('str')

In [59]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="geoapiExercises")
#listing_data['city_geopy'] = listing_data.apply(lambda row: set_city(row['lat'], row['lng'], geolocator), axis=1)
print(set_city(35, -76, geolocator))


---Latitude casted to string at 0.0 seconds ---
---Longitude casted at 0.0 seconds ---
---Get location json from geopy at 0.34982752799987793 seconds ---
---Get address json at 0.34982752799987793 seconds ---
---Get city value at 0.34982752799987793 seconds ---



In [6]:
price_data_grouped = price_data.groupby('id').mean()

In [7]:
list_n_price_data = listing_data.merge(price_data_grouped, how = 'left', on='id')

In [ ]:
# Future way to pull down listing data
"""
path = 'C:/Users/mattg/Desktop/Hobbies/bucket_data'
df = pd.DataFrame(columns=['observation'])
for directory in os.listdir(path):
    if os.path.isdir(directory):
        for filename in os.listdir(directory):
            with open(os.path.join(directory, filename)) as f:
                observation = f.read()
                print('hi')
                current_df = pd.DataFrame({'observation': [observation]})
                df = df.append(current_df, ignore_index=True)
"""

In [ ]:
from pandas.plotting import scatter_matrix
scatter_matrix(listing_data, alpha=0.2, figsize=(15,15))

In [ ]:
#null_town = all_ne_data[all_ne_data['town'].isna()]
#null_town.to_csv('nulltowns.csv')

In [ ]:
#bed_bath_prices = all_ne_data.groupby(['beds', 'baths'])['price'].mean().reset_index()
#bed_bath_counts = all_ne_data.groupby(['beds', 'baths'])['price'].count().reset_index()
bed_bath_prices = pd.read_csv('data/bed_bath_avgs_wbeds.csv')
bed_bath_counts = pd.read_csv('data/bed_bath_counts_wbeds.csv')


In [15]:

bed_bath_prices_rev2 = listing_data.groupby(['beds_no', 'baths_no', 'baths_type'])['price'].mean().reset_index()
bed_bath_counts_rev2 = listing_data.groupby(['beds_no', 'baths_no', 'baths_type'])['price'].count().reset_index()
bed_bath_prices_rev2 = bed_bath_prices_rev2.dropna(axis=0, subset=['beds_no'])
bed_bath_counts_rev2 = bed_bath_counts_rev2.dropna(axis=0, subset=['beds_no'])
bed_bath_prices_rev2.to_csv('grouped_data/bed_bath_prices_rev2.csv')
bed_bath_prices_rev2.to_csv('grouped_data/bed_bath_counts_rev2.csv')


In [ ]:
bed_bath_prices_rev2 = pd.read_csv('grouped_data/bed_bath_prices_rev2.csv')
bed_bath_counts_rev2 = pd.read_csv('grouped_data/bed_bath_counts_rev2.csv')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.scatterplot(x=bed_bath_prices_rev2['beds_no'], y=bed_bath_prices_rev2['baths_no'], style=bed_bath_prices_rev2['baths_type'], hue=bed_bath_prices_rev2['price'])
plt.title('Price vs Bed/Bath Combinations')
plt.xlabel('Baths')
plt.ylabel('Beds')

In [ ]:
avgs = listing_data.groupby(['guest_no','Region'])['price'].mean().reset_index()
import seaborn as sns
sns.scatterplot(x=avgs['guest_no'], y=avgs['price'], hue = avgs['Region'])
plt.title('Price vs Guests')
plt.xlabel('Guests')
plt.ylabel('Avg Price')


In [ ]:
avgRats = listing_data.groupby(['guest_no','Region'])['avgRating'].mean().reset_index()
sns.scatterplot(x=avgRats['guest_no'], y=avgRats['avgRating'], hue = avgRats['Region'])
plt.title('Avg Rating vs Guests')
plt.xlabel('Guests')
plt.ylabel('Avg Rating')


In [ ]:
sns.boxplot(x="guest_no", y="price", data=listing_data)
plt.ylim([0,2000])

In [ ]:
counts = listing_data.groupby(['guest_no'])['id'].count().reset_index()
sns.scatterplot(x=counts['guest_no'], y=counts['id'])
plt.title('Count of IDs vs Guests')
plt.xlabel('# of Guests')
plt.ylabel('ID Count')

In [ ]:
counts_loc = listing_data.groupby(['guest_no','Region'])['id'].count().reset_index()
sns.scatterplot(x=counts_loc['guest_no'], y=counts_loc['id'], hue = counts_loc['Region'])
plt.title('Count of IDs vs Guests')
plt.xlabel('# of Guests')
plt.ylabel('ID Count')

In [ ]:
sns.boxplot(x="guest_no", y="avgRating", data=listing_data)
#plt.ylim([0,2000])

In [ ]:
listing_data.columns

In [ ]:
import folium
#listing_locations = all_ne_data[["lat", "lng", "name"]]
#listing_locations = listing_locations.dropna()
listing_data = listing_data.dropna()
m = folium.Map(location=[listing_data.lat.mean(), listing_data.lng.mean()], zoom_start=3)

tooltip = "Click me!"
counter = 0
for index, location_info in listing_data.iterrows():
#for index, location_info in listing_locations.iterrows():
    if location_info.reviewsCount > 100:
        folium.Marker(
            [location_info["lat"], location_info["lng"]], popup=location_info["price"], icon=folium.Icon(color="darkgreen")
        ).add_to(m)
    else:
        folium.Marker(
            [location_info["lat"], location_info["lng"]], popup=location_info["price"], icon=folium.Icon(color="blue")
        ).add_to(m)        
    counter += 1
    if counter == 1000:
        break

#m.save('map.html')

In [ ]:
m

In [ ]:
listing_data.isSuperhost.unique()

In [22]:
listing_data.shape

(35284, 30)

In [ ]:
listing_data

In [ ]:
nc = listing_data[listing_data['top_lat'] < 35.5]

In [ ]:
nc.shape

In [ ]:
nc.town.unique()

In [ ]:
from glob import glob
import numpy as np
import folium
from folium import plugins
from folium.plugins import HeatMap

lon, lat = -77.5, 34.2
zoom_start = 5

#data = (
#    np.random.normal(size=(100, 3)) *
#    np.array([[1, 1, 1]]) +
#    np.array([[48, 5, 1]])
#).tolist()
m = folium.Map([48, 5], tiles='stamentoner', zoom_start=6)
#nc = nc.dropna()
HeatMap(listing_data[['lat','lng']].groupby(['lat','lng']).sum().reset_index().values.tolist(), radius=7, max_zoom=10).add_to(folium.FeatureGroup(name='Heat Map').add_to(m))
folium.LayerControl().add_to(m)

In [ ]:
m

In [82]:
import plotly.express as px
listing_fig = px.density_mapbox(listing_data, lat='lat', lon='lng', z='id', radius=10,
                        center=dict(lat=listing_data.lat.mean(), lon=listing_data.lng.mean()), zoom=3,
                        mapbox_style="stamen-terrain")
#fig.show()
listing_fig.update_layout(title_text="Heatmap for Airbnb Listings")

listing_fig.write_html("reports/listing_heatmap.html")

In [83]:
price_fig = px.density_mapbox(listing_data, lat='lat', lon='lng', z='price', radius=10,
                        center=dict(lat=listing_data.lat.mean(), lon=listing_data.lng.mean()), zoom=3,
                        mapbox_style="stamen-terrain")
#fig.show()
price_fig.update_layout(title_text="Heatmap for Airbnb Price")
price_fig.write_html("reports/price_heatmap.html")

In [90]:
with open("reports/report_draft.html",'w') as f:
    f.write(listing_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(price_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(price_v_numlist_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(roi_fig.to_html(full_html=False, include_plotlyjs='cdn'))
    f.write(roi_subplots.to_html(full_html=False, include_plotlyjs='cdn'))

In [1]:
pip install us

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 132.6/132.6 KB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Using legacy 'setup.py install' for us, since package 'wheel' is not installed.
Using legacy 'setup.py install' for jellyfish, since package 'wheel' is not installed.
  Running setup.py install for jellyfish: started
  Running setup.py install for jellyfish: finished with status 'done'
  Running setup.py install for us: started
  Running setup.py install for us: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


In [10]:
city_data = pd.read_csv('resource_data/uscities.csv')

In [47]:
city_data.columns

Index(['city', 'city_ascii', 'state_id', 'state_name', 'county_fips',
       'county_name', 'lat', 'lng', 'population', 'density', 'source',
       'military', 'incorporated', 'timezone', 'ranking', 'zips', 'id'],
      dtype='object')

In [20]:
test_zip = 91311
city_name = 'Los Angeles'
    zip_codes = city_data[city_data['city'] == city_name].zips

1        91367 90291 90293 90292 91316 91311 90035 9003...
23721                                                78580
Name: zips, dtype: object

In [11]:
home_value = pd.read_csv('resource_data/home_value_zillow.csv')

In [12]:
city_home = city_data.merge(home_value, left_on = ['city','state_id'], right_on = ['RegionName','State'])

In [54]:
city_home

,city,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,...,2021-07-31,2021-08-31,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30
0,New York,New York,NY,New York,36081,Queens,40.6943,-73.9249,18680025,10768.0,...,716872.0,721839.0,727737.0,732735.0,736427.0,739231.0,743318.0,748195.0,754331.0,759901.0
1,Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1141,-118.4068,12531334,3267.0,...,901800.0,915560.0,925407.0,928293.0,932714.0,938376.0,943426.0,952509.0,968695.0,988288.0
2,Chicago,Chicago,IL,Illinois,17031,Cook,41.8375,-87.6866,8586888,4576.0,...,295982.0,299067.0,301141.0,303191.0,305794.0,308245.0,310569.0,311893.0,313825.0,315196.0
3,Miami,Miami,FL,Florida,12086,Miami-Dade,25.7840,-80.2101,6076316,4945.0,...,415291.0,422454.0,430923.0,439804.0,447950.0,456631.0,466100.0,475103.0,484963.0,494527.0
4,Dallas,Dallas,TX,Texas,48113,Dallas,32.7935,-96.7667,5910669,1522.0,...,270708.0,277608.0,284215.0,289697.0,292707.0,295313.0,299936.0,306606.0,313368.0,321569.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18376,Belvidere,Belvidere,TN,Tennessee,47051,Franklin,35.1281,-86.1881,8,6.0,...,192853.0,196955.0,200832.0,204849.0,208177.0,211980.0,216874.0,221173.0,224086.0,226282.0
18377,Monarch,Monarch,MT,Montana,30013,Cascade,47.1003,-110.8448,5,2.0,...,227330.0,237445.0,245249.0,252902.0,258049.0,260623.0,265979.0,269531.0,273999.0,275404.0
18378,Lake Buena Vista,Lake Buena Vista,FL,Florida,12095,Orange,28.3790,-81.5262,4,338.6,...,3656825.0,3763472.0,3858074.0,3927890.0,4006278.0,4076415.0,4159371.0,4237784.0,4344569.0,4476184.0
18379,Harwood,Harwood,MO,Missouri,29217,Vernon,37.9567,-94.1541,3,59.7,...,NaN,NaN,NaN,62534.0,61983.0,63615.0,65138.0,67367.0,68567.0,69566.0


In [28]:
city_home.shape

(75322, 293)

In [92]:
city_home['city'] = city_home['city'].str.lower()
lists_per_city = listing_data.groupby(['town'])['id'].count().reset_index()
lists_per_city.rename(columns = {'id':'num_lists_in_town'}, inplace = True)
median_price_per_city = listing_data.groupby(['town'])['price'].median().reset_index()
median_price_per_city.rename(columns = {'price':'median_price_per_city'}, inplace = True)

# Calculate occupancy rate
occ_rate = occ_data.groupby('id')['available'].apply(lambda row: np.sum(row)/len(row))
listing_occ_rate = listing_data.merge(occ_rate, on = 'id')
median_occ_rate_per_city = listing_occ_rate.groupby(['town'])['available'].median().reset_index()
median_occ_rate_per_city.rename(columns = {'available':'occupancy_rate'}, inplace = True)

# Calculate pricing avgs
cleaning_fee = price_data.groupby(['id'])['cleaning_fee'].median().reset_index()
service_fee = price_data.groupby(['id'])['service_fee'].median().reset_index()
total_price = price_data.groupby(['id'])['total_price'].median().reset_index()


In [14]:
lists_town_price = lists_per_city.merge(median_price_per_city, left_on = 'town', right_on = 'town')
lists_town_price_occ = lists_town_price.merge(median_occ_rate_per_city, left_on = 'town', right_on = 'town')
lists_town_home_price_occ = lists_town_price_occ.merge(city_home, left_on = 'town', right_on = 'city')

In [91]:
price_data.head(5)

,index,cleaning_fee,service_fee,total_price,check_in,check_out,total_price_description,currency,pull_time,id,parition_col
0,0,129.0,46.0,199.0,2022-04-18,2022-04-19,$199 x 1 night,$,18/04/2022 23:52:23,41935100.0,0.0
1,0,NaN,39.0,275.0,2022-04-18,2022-04-19,$275 x 1 night,$,18/04/2022 23:52:38,41508820.0,0.0
2,0,129.0,46.0,199.0,2022-04-19,2022-04-20,$199 x 1 night,$,18/04/2022 23:52:48,41935100.0,0.0
3,0,NaN,39.0,275.0,2022-04-19,2022-04-20,$275 x 1 night,$,18/04/2022 23:52:55,41508820.0,0.0
4,0,129.0,46.0,199.0,2022-04-20,2022-04-21,$199 x 1 night,$,18/04/2022 23:53:02,41935100.0,0.0


In [57]:
newbury_example = city_home_airbnblist_price[city_home_airbnblist_price['city'] == 'newbury']
newbury_example.to_csv('newbury.csv')

In [15]:
lists_town_home_price_occ_vt = lists_town_home_price_occ[(lists_town_home_price_occ['state_id'] == 'VT')]
lists_town_home_price_occ_nh = lists_town_home_price_occ[(lists_town_home_price_occ['state_id'] == 'NH')]
lists_town_home_price_occ_nc = lists_town_home_price_occ[(lists_town_home_price_occ['state_id'] == 'NC')]
lists_town_home_price_occ_sc = lists_town_home_price_occ[(lists_town_home_price_occ['state_id'] == 'SC')]

lists_town_home_price_occ_all = lists_town_home_price_occ_vt.append(lists_town_home_price_occ_nh)
lists_town_home_price_occ_all = lists_town_home_price_occ_all.append(lists_town_home_price_occ_nc)
lists_town_home_price_occ_all = lists_town_home_price_occ_all.append(lists_town_home_price_occ_sc)


In [121]:
lists_town_home_price_occ_all

,town,num_lists_in_town,median_price_per_city,occupancy_rate,city,city_ascii,state_id,state_name,county_fips,county_name,...,2021-07-31,2021-08-31,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30
58,barre,46,115.0,0.892603,barre,Barre,VT,Vermont,50023,Washington,...,222827.0,224359.0,228725.0,232781.0,236740.0,240078.0,245099.0,250053.0,255472.0,255168.0
106,bolton,6,137.5,0.437808,bolton,Bolton,VT,Vermont,50007,Chittenden,...,277040.0,282711.0,285847.0,293143.0,298243.0,304080.0,305394.0,307785.0,310612.0,315936.0
199,chittenden,15,249.0,0.476712,chittenden,Chittenden,VT,Vermont,50021,Rutland,...,326685.0,331385.0,336711.0,342811.0,346676.0,349019.0,350501.0,354296.0,357322.0,360709.0
239,danby,10,272.5,0.527397,danby,Danby,VT,Vermont,50021,Rutland,...,290377.0,293850.0,301736.0,308725.0,307010.0,300192.0,292006.0,294239.0,297632.0,300998.0
420,grafton,27,176.0,0.829041,grafton,Grafton,VT,Vermont,50025,Windham,...,387753.0,383065.0,382901.0,393556.0,394396.0,395337.0,396719.0,402751.0,408554.0,415550.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
942,troy,6,169.5,0.408219,troy,Troy,SC,South Carolina,45047,Greenwood,...,80869.0,82541.0,84666.0,88649.0,91523.0,96403.0,98923.0,100756.0,99626.0,98450.0
950,wallace,4,104.5,0.180274,wallace,Wallace,SC,South Carolina,45069,Marlboro,...,61076.0,61955.0,62766.0,63719.0,65087.0,67673.0,69428.0,70971.0,69539.0,68445.0
1014,williston,14,133.5,0.443836,williston,Williston,SC,South Carolina,45011,Barnwell,...,127116.0,127429.0,128398.0,129386.0,130987.0,133448.0,135737.0,138493.0,140110.0,142649.0
1031,windsor,27,127.0,0.643836,windsor,Windsor,SC,South Carolina,45003,Aiken,...,153242.0,153236.0,154587.0,155441.0,158788.0,162620.0,166052.0,168770.0,170462.0,175491.0


In [77]:
import plotly.graph_objects as go

price_v_numlist_fig = go.Figure(data=[
    go.Bar(name='Price', x=lists_town_home_price_occ_all['town'], y=lists_town_home_price_occ_all['median_price_per_city']),
    go.Bar(name='# of Listings', x=lists_town_home_price_occ_all['town'], y=lists_town_home_price_occ_all['num_lists_in_town'])
])
# Change the bar mode
price_v_numlist_fig.update_layout(title_text='Price vs Number of Airbnb Listings', barmode='group')
price_v_numlist_fig.show()

In [125]:
print(calculate_mortgage(100000, 3, 15))

690.58


In [17]:
lists_town_home_price_occ_all['avg_30_yr_mort'] = lists_town_home_price_occ_all.apply(lambda row: calculate_mortgage(row['2022-04-30'], 5, 30), axis=1)

In [18]:
lists_town_home_price_occ_all['median_ROI'] = lists_town_home_price_occ_all.apply(lambda row: calculate_roi(row['median_price_per_city'], row['occupancy_rate'], row['avg_30_yr_mort']), axis=1)

In [64]:
lists_town_home_price_occ_all.head(5)

,town,num_lists_in_town,median_price_per_city,occupancy_rate,city,city_ascii,state_id,state_name,county_fips,county_name,...,2021-09-30,2021-10-31,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,avg_30_yr_mort,median_ROI
58,barre,46,115.0,0.892603,barre,Barre,VT,Vermont,50023,Washington,...,228725.0,232781.0,236740.0,240078.0,245099.0,250053.0,255472.0,255168.0,1369.80,1.279347
106,bolton,6,137.5,0.437808,bolton,Bolton,VT,Vermont,50007,Chittenden,...,285847.0,293143.0,298243.0,304080.0,305394.0,307785.0,310612.0,315936.0,1696.01,0.079617
199,chittenden,15,249.0,0.476712,chittenden,Chittenden,VT,Vermont,50021,Rutland,...,336711.0,342811.0,346676.0,349019.0,350501.0,354296.0,357322.0,360709.0,1936.36,0.864581
239,danby,10,272.5,0.527397,danby,Danby,VT,Vermont,50021,Rutland,...,301736.0,308725.0,307010.0,300192.0,292006.0,294239.0,297632.0,300998.0,1615.82,1.705347
420,grafton,27,176.0,0.829041,grafton,Grafton,VT,Vermont,50025,Windham,...,382901.0,393556.0,394396.0,395337.0,396719.0,402751.0,408554.0,415550.0,2230.76,0.989516


In [78]:
import plotly.graph_objects as go

roi_fig = go.Figure(data=[
    go.Bar(name='ROI', x=lists_town_home_price_occ_all['town'], y=lists_town_home_price_occ_all['median_ROI']),
])
# Change the bar mode
roi_fig.update_layout(title_text = 'ROI by City', barmode='group')
roi_fig.show()

In [132]:
lists_town_home_price_occ_all['median_ROI'].sort_values(ascending=False)

635    10.500597
189    10.400822
802    10.285397
560     9.588095
221     9.218663
         ...    
458    -0.752642
685    -0.783152
631    -0.877034
364    -1.000000
358    -1.000000
Name: median_ROI, Length: 141, dtype: float64

In [89]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

roi_subplots = make_subplots(rows=2, cols=2)

lists_town_home_price_occ_vt = lists_town_home_price_occ_all[(lists_town_home_price_occ_all['state_id'] == 'VT')]
lists_town_home_price_occ_nh = lists_town_home_price_occ_all[(lists_town_home_price_occ_all['state_id'] == 'NH')]
lists_town_home_price_occ_nc = lists_town_home_price_occ_all[(lists_town_home_price_occ_all['state_id'] == 'NC')]
lists_town_home_price_occ_sc = lists_town_home_price_occ_all[(lists_town_home_price_occ_all['state_id'] == 'SC')]

roi_subplots.append_trace(go.Bar(
    x=lists_town_home_price_occ_vt['town'],
    y=lists_town_home_price_occ_vt['median_ROI'], name="VT"
), row=1, col=1)

roi_subplots.append_trace(go.Bar(
    x=lists_town_home_price_occ_nh['town'],
    y=lists_town_home_price_occ_nh['median_ROI'], name="NH"
), row=1, col=2)

roi_subplots.append_trace(go.Bar(
    x=lists_town_home_price_occ_nc['town'],
    y=lists_town_home_price_occ_nc['median_ROI'], name="NC"
), row=2, col=1)

roi_subplots.append_trace(go.Bar(
    x=lists_town_home_price_occ_sc['town'],
    y=lists_town_home_price_occ_sc['median_ROI'], name="SC"
), row=2, col=2)


roi_subplots.update_layout(height = 1100, title_text="ROI for Cities in each State", barmode='group')
roi_subplots.show()

In [ ]:

    
    
# include price viz's next


